In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # Loads env from .env file

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("Missing GROQ_API_KEY")



In [ ]:
from groq import Groq
client = Groq() # Loads the API key automatically from the environment variable
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": "Explain why fast inference is critical for reasoning models"
        }
    ]
)
print(completion.choices[0].message.content)


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
import subprocess
from pathlib import Path

def clone_repo(repo_url: str, target_dir: str):
    subprocess.run(["git", "clone", "--depth", "1", repo_url, target_dir], check=True)
    tempPath = Path(target_dir)
    gitPath = tempPath / ".git"
    subprocess.run(["rm", "-rf", gitPath], check=True)

In [2]:
def read_file_to_string(file_path: str) -> str:
    path = Path(file_path)
    if not path.is_file():
        raise FileNotFoundError(f"{file_path} is not a file.")
    return path.read_text(encoding="utf-8", errors="ignore")

In [3]:
def remove_first_folder(file_path: str) -> str:
    parts = Path(file_path).parts
    if len(parts) <= 1:
        return file_path  # Nothing to remove
    return str(Path(*parts[1:]))

In [4]:
def read_dir_rec(dir_path):
    files = dict()
    from pathlib import Path
    root = Path(dir_path)
    for child in root.iterdir():
        if child.is_file():
            files[remove_first_folder(root / child.name)] = read_file_to_string(root / child.name)
            print(f"File: {root / child.name}")
        elif child.is_dir():
            print(f"Directory: {child.name}")
            files = files | read_dir_rec(root / child.name)
    return files

In [5]:

from llama_index.core.schema import TextNode
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.core.indices import VectorStoreIndex, load_index_from_storage

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
# Set the embedding model, we do this only once when we start the backend

In [ ]:
def create_index(files_dict):

    # Convert dict into list, concatenate file names and contents into one string
    documents = [
        {
            "path": name,
            "text": f"FILE PATH: \n{name}, \nFILE CONTENT: \n{content}"
        }
        for name, content in files_dict.items()
    ]

    # Load documents into textnodes
    text_nodes = []
    for d in documents:
        new_node = TextNode(text=d["text"], metadata={"path" : d["path"]})
        text_nodes.append(new_node)

    # Create the index
    vector_store = SimpleVectorStore() # TODO maybe use a more capable vector store like qdrant? 
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    llm = Ollama(model="gemma3:4b", request_timeout=300)#, base_url="http://172.26.44.37:11434") # 

    index = VectorStoreIndex(text_nodes)

    # Store index in directory
    index.storage_context.persist(persist_dir="./index")

In [36]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="gemma3:4b-q4_K_M", request_timeout=300, stream=True)#, base_url="http://172.0.0.1:11434", stream=True)

response = llm.complete("Say hello world.")
for chunk in response:
    print(chunk.text, end="")



ResponseError: model 'gemma3:4b-q4_K_M' not found (status code: 404)

In [23]:
def query_index():
    storage_context = StorageContext.from_defaults(persist_dir="./index")
    index = load_index_from_storage(storage_context)
    query="What are the methods that make up the genetic algorithm?"
    query = "How did they center the div?"
    query = "how does the game loop work?"

    retriever_engine = index.as_retriever(similarity_top_k=10)
    retrieval_results = retriever_engine.retrieve(query)
    retrieved_drawing_ids = [n.node.metadata["path"] for n in retrieval_results]
    print(retrieved_drawing_ids)

In [16]:
def clone_and_read(repo_url):
    clone_repo(repo_url=repo_url, target_dir="./temp")
    parsed_files = read_dir_rec("./temp")
    print(parsed_files)
    create_index(parsed_files)
    query_index() 
    subprocess.run(["rm", "-rf", "./temp"])

In [24]:
clone_and_read("https://github.com/itsmejul/flappy-evolve")

Cloning into './temp'...


Directory: src
File: temp/src/pipe.js
File: temp/src/main.js
File: temp/src/bird.js
File: temp/src/consts.js
File: temp/src/mlp.js
File: temp/src/genetic_utils.js
File: temp/index.html
File: temp/style.css
File: temp/README.md
{'src/pipe.js': 'const pipeWidth = 50;\nconst pipeGap = 120; \nconst spawnMargin = 20; // Margin on top/bottom to make sure pipe gap is not at the very edge of the canvas\n\nexport class Pipe {\n  constructor(gapY, canvas) {\n    this.x = canvas.width;\n    this.gapY = gapY;\n    this.width = pipeWidth;\n    this.gap = pipeGap;\n  }\n\n  draw(ctx, canvas) {\n    ctx.fillStyle = "green";\n    // Top part of pipe\n    ctx.fillRect(this.x, 0, this.width, this.gapY);\n    // Bottom part of pipe\n    ctx.fillRect(\n      this.x,\n      this.gapY + this.gap,\n      this.width,\n      canvas.height - (this.gapY + this.gap)\n    );\n  }\n\n  // create a new pipe with random gap\n  static spawn(canvas) {\n    const minY = spawnMargin;\n    const maxY = canvas.height - pip